In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import BatchNormalization , Dense , Dropout , GlobalAveragePooling2D

In [ ]:
def createEffNetB0(input_shape):
    model = EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=input_shape,)
    return model
basemodel = createEffNetB0((105, 105, 3))
basemodel.trainable = False
def create_model(base_model):
    
    x =  BatchNormalization()(base_model.output)
    x =  GlobalAveragePooling2D()(x)
    
    # Font Family
    dense_family = Dense(1024,activation = 'relu')(x)
    dropout_family = Dropout(0.2)(dense_family)
    output_family = Dense(7,activation = 'softmax')(dropout_family)
 
    model = tf.keras.Model(inputs = base_model.input , outputs = output_family)
    
    return model
model = create_model(basemodel)

In [ ]:
# Check for input path!
#model.load_weights('../input/font-regcog-inception/inception_best.h5')
model.load_weights('../input/fontsize-weight/eff_size_best.h5')

In [ ]:
model.compile(optimizer =  tf.keras.optimizers.Adam(),
              # กำหนด loss แบบ list ตาม order ที่create model or use 1 for all
            loss = [tf.keras.losses.SparseCategoricalCrossentropy()],
            metrics = ['acc']
            )

In [ ]:
id_test = pd.read_csv('../input/font-detection-dataset-crop/label_test.csv')
id_test['id'] = id_test['id'] + '.png'

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
test_datagen = ImageDataGenerator(preprocessing_function = tf.keras.applications.efficientnet.preprocess_input)


test_generator = test_datagen.flow_from_dataframe(
     dataframe= id_test[:], # [:] for full data
     directory="../input/font-detection-dataset-crop/image_test",
     target_size=(105, 105),
     x_col = "id",
     shuffle=False,
     seed=69,
     batch_size=128,
     class_mode = None)

In [ ]:
pred = model.predict(test_generator)

In [ ]:
metadata_aux = pd.read_csv('../input/font-detection-dataset-crop/label_train.csv')
# map Fontfamily to interger
unique_vals = metadata_aux['fontSize'].unique()
metadata_aux['fontSize_int'] = metadata_aux['fontSize'].replace(to_replace=unique_vals,
           value= list(range(len(unique_vals))))

In [ ]:
def map_interger_to_label(mode = None, integer = None):
    if mode == 'family':
        return metadata_aux[metadata_aux['fontFamily_int'] == integer]['fontFamily'].sample().to_string().split(' ')[-1]
    elif mode == 'weight':
        return metadata_aux[metadata_aux['fontWeight_int'] == integer]['fontWeight'].sample().to_string().split(' ')[-1]
    elif mode == 'style':
        return metadata_aux[metadata_aux['fontStyle_int'] == integer]['fontStyle'].sample().to_string().split(' ')[-1]
    elif mode == 'size':
        return metadata_aux[metadata_aux['fontSize_int'] == integer]['fontSize'].sample().to_string().split(' ')[-1]

In [ ]:
from tqdm.notebook import tqdm as tqdm
test_label = {}
for i in tqdm(range(len(id_test))):
    prediction = None
    parentId = id_test.loc[i].to_string().split(' ')[-1].split('.')[0]
    # Get font family prediction
    prediction = np.argmax(pred[i])
    prediction = map_interger_to_label(mode = 'size', integer = prediction)
    test_label[parentId] =  prediction

In [ ]:
sample_submission_df = pd.read_csv('../input/testsubmitcontinue/sample_submission1.csv')
sample_submission_df.set_index('Id',inplace=True)

In [ ]:
# map output to submission.csv
for idx , label in tqdm(test_label.items()):
    doc_id = idx
    #family = label
    #style = label['Style']
    #weight = label['Weight']
    size = label
    
    
    #sample_submission_df.loc[f'{idx}_name'] = family
    #sample_submission_df.loc[f'{idx}_file'] = f'{family}.ttf'.replace('_',' ')
    sample_submission_df.loc[f'{idx}_size'] = size
    #sample_submission_df.loc[f'{idx}_style-weight'] = f'{style}-{weight}'


In [ ]:
sample_submission_df.to_csv('./sample_submission.csv')